In [ ]:
!pip install -q transformers peft bitsandbytes accelerate trl datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 15.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.7/564.7 kB 30.2 MB/s eta 0:00:00


In [ ]:
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    TrainingArguments,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer
from google.colab import userdata
import os

In [ ]:
hf_token = userdata.get('HF_TOKEN')
os.environ["HF_TOKEN"] = hf_token
!huggingface-cli login --token $HF_TOKEN

⚠️  Warning: 'huggingface-cli login' is deprecated. Use 'hf auth login' instead.
The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `hf`CLI if you want to set the git credential as well.
Token is valid (permission: write).
The token `lab_calculator` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [ ]:
train_dataset = load_dataset('json', data_files='train.jsonl', split='train')
eval_dataset = load_dataset('json', data_files='eval.jsonl', split='train')

Generating train split: 0 examples [00:00, ? examples/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
def format_prompt(example):
    # This function creates a structured prompt for each data point
    # The model is trained to generate the text that comes after "### CODE:"
    prompt = (
        f"### CONTEXT:\n{example['context']}\n\n"
        f"### OBSERVATIONS:\n{example['observations']}\n\n"
        f"### CODE:\n{example['code']}"
    )
    return prompt

In [ ]:
model_name = "microsoft/Phi-3-mini-4k-instruct"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=False,
)

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    trust_remote_code=True,
    device_map="auto"
)
model.config.use_cache = False
model.config.pretraining_tp = 1

tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

config.json:   0%|          | 0.00/967 [00:00<?, ?B/s]

configuration_phi3.py: 0.00B [00:00, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-4k-instruct:
- configuration_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


modeling_phi3.py: 0.00B [00:00, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/microsoft/Phi-3-mini-4k-instruct:
- modeling_phi3.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


...

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/306 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/599 [00:00<?, ?B/s]

In [ ]:
peft_config = LoraConfig(
    lora_alpha=32,
    lora_dropout=0.05,
    r=16,
    bias="none",
    task_type="CAUSAL_LM",
    target_modules=[
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj"
    ]
)

In [ ]:
training_arguments = TrainingArguments(
    output_dir="./phi3-lab-report-coder",
    num_train_epochs=10,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    optim="paged_adamw_32bit",
    save_steps=50,
    logging_steps=10,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=True,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="constant",
    report_to="tensorboard",
    eval_steps=50,
    do_eval=True,
)

In [ ]:
trainer = SFTTrainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    peft_config=peft_config,
    args=training_arguments,
    formatting_func=format_prompt,
)

/usr/local/lib/python3.12/dist-packages/peft/mapping_func.py:73: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/peft/tuners/tuners_utils.py:196: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


In [ ]:
print("🚀 Starting model training...")
trainer.train()

🚀 Starting model training...


Step,Training Loss
10,0.685500


Step,Training Loss
10,0.685500
20,0.553300
30,0.453000
40,0.411700
50,0.331100
60,0.267600
70,0.213400
80,0.156500
90,0.123300
100,0.101400


TrainOutput(global_step=100, training_loss=0.3296881079673767, metrics={'train_runtime': 2550.6088, 'train_samples_per_second': 0.145, 'train_steps_per_second': 0.039, 'total_flos': 6369432681369600.0, 'train_loss': 0.3296881079673767, 'epoch': 10.0})

In [ ]:
final_model_path = "./phi3-lab-report-coder-final"
trainer.save_model(final_model_path)
print(f"✅ Training complete! Model saved to {final_model_path}")

✅ Training complete! Model saved to ./phi3-lab-report-coder-final


In [ ]:

# --- 2. Imports ---
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel
import json

# --- 3. Configuration ---
# Define the model IDs and paths
base_model_id = "microsoft/Phi-3-mini-4k-instruct"
# IMPORTANT: This path should point to where your final adapter was saved.
# In your training script, it was saved to "./phi3-finetuned-adapter"
adapter_path = "./phi3-lab-report-coder-final"

# --- 4. Load the Fine-Tuned Model and Tokenizer ---

# Configure the quantization settings (must match the settings used during training)
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=True,
)

# Load the base Phi-3 model with the specified quantization config
print(f"Loading base model: {base_model_id}")
base_model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    trust_remote_code=True,
    quantization_config=bnb_config,
    device_map="auto", # Automatically map layers to available devices (e.g., GPU)
)

# Load the tokenizer for the base model
tokenizer = AutoTokenizer.from_pretrained(base_model_id, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token # Set the pad token to the end-of-sentence token
tokenizer.padding_side = "right" # Pad on the right side for generation

# Load the LoRA adapter and merge it with the base model to create the fine-tuned model
print(f"Loading LoRA adapter from: {adapter_path}")
model = PeftModel.from_pretrained(base_model, adapter_path)
print("✅ LoRA adapter loaded and merged successfully!")

# Set the model to evaluation mode for inference
model.eval()

# --- 5. Run Inference ---

# Define a test case with context and observations
test_context = "An experiment to verify Ohm's Law and determine the unknown resistance (R). The formula is V = IR. The result should be the average resistance from all trials."
test_observations = {
    "unit_V": "volts",
    "unit_I": "amperes",
    "readings": [
      {"V": 2.0, "I": 0.21},
      {"V": 4.0, "I": 0.40},
      {"V": 6.0, "I": 0.62}
    ]
}

# Format the prompt exactly as it was during training, but without the final 'code' part
prompt_for_inference = (
    f"### CONTEXT:\n{test_context}\n\n"
    f"### OBSERVATIONS:\n{json.dumps(test_observations, indent=2)}\n\n" # Using json.dumps for clean formatting
    f"### CODE:\n"
)


# Tokenize the input prompt and move it to the model's device
inputs = tokenizer(prompt_for_inference, return_tensors="pt").to(model.device)

print("\n--- GENERATING CODE ---")

# Generate the output from the model
# Using slightly different parameters optimized for inference
generated_output = model.generate(
    **inputs,
    max_new_tokens=256,      # Maximum number of new tokens to generate
    do_sample=True,          # Enable sampling for more creative outputs
    top_p=0.9,               # Use nucleus sampling
    temperature=0.3,         # Lower temperature for more focused and less random code
    use_cache=False,          # Enable caching for faster generation
    pad_token_id=tokenizer.eos_token_id # Set pad token to avoid warnings
)

# Decode the generated tokens, skipping the original prompt to get only the new code
input_length = inputs["input_ids"].shape[1]
generated_text = tokenizer.decode(generated_output[0][input_length:], skip_special_tokens=True)

# --- 6. Print the Results ---
print("\n" + "="*50)
print("PROMPT SENT TO MODEL:")
print("="*50)
print(prompt_for_inference)

print("\n" + "="*50)
print("GENERATED CODE:")
print("="*50)
print(generated_text)





Loading base model: microsoft/Phi-3-mini-4k-instruct


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading LoRA adapter from: ./phi3-lab-report-coder-final
✅ LoRA adapter loaded and merged successfully!

--- GENERATING CODE ---

PROMPT SENT TO MODEL:
### CONTEXT:
An experiment to verify Ohm's Law and determine the unknown resistance (R). The formula is V = IR. The result should be the average resistance from all trials.

### OBSERVATIONS:
{
  "unit_V": "volts",
  "unit_I": "amperes",
  "readings": [
    {
      "V": 2.0,
      "I": 0.21
    },
    {
      "V": 4.0,
      "I": 0.4
    },
    {
      "V": 6.0,
      "I": 0.62
    }
  ]
}

### CODE:


GENERATED CODE:
def calculate_resistance(V, I):
    if I == 0:
        return float('inf') # Avoid division by zero
    return V / I

# Observations from the experiment
observations = [
    {"V": 2.0, "I": 0.21},
    {"V": 4.0, "I": 0.40},
    {"V": 6.0, "I": 0.62}
]

# Calculate resistance for each reading
resistance_values = [calculate_resistance(r['V'], r['I']) for r in observations]

# Calculate average resistance
average_resistance =